In [ ]:
import simulator_setup as sim

In [ ]:
depth = 2
number_of_witnesses_per_depth = [2, 2, 2]
threshold = 1.0

prob_honest = 0.5
prob_coerced = 0.5


environment_size = [(0, 2), (0, 2)]
number_of_cars = 1000

width = environment_size[0][1] - environment_size[0][0]
height = environment_size[1][1] - environment_size[1][0]
density = number_of_cars / (width * height)

number_of_simulations = 1



In [ ]:
cars, location_adapter, containers = sim.simulator_setup(environment_size, number_of_cars, prob_coerced, prob_honest)

#sim_df = sim.simulator_caller(number_of_simulations, number_of_cars, prob_coerced, prob_honest, depth,
                            #cars, containers, location_adapter, 
                            #number_of_witnesses_per_depth, density, threshold, environment_size)
#print('here')

In [ ]:
from t_pop.algorithms.tree import Tree, get_neighbours_dict
from t_pop.collections.components.containers import Containers
from t_pop.collections.components.location_cache import LocationCacheType


import random

neighbours_dictionary = get_neighbours_dict(cars[128], location_adapter)
print(neighbours_dictionary.values())
print('finish')

def witness_dictionary_gen(neighbours_dictionary:dict, number_of_witnesses:int) -> dict:
    """
    Returns a dictionary of witnesses, with the witness ID as key and its location cache as value

    If there is only a true location cache, sample both your witnesses from true location cache
    if there is only fake location cache, sample both your witnesses from your fake location cache
    If you have both true and fake location cache: samples from both and then again to get to the right size

    :param neighbours dictionary: the car whose range of sight is to be checked
    :return: witness dictionary
    """
    witnesses_dictionary = {}
    # each key is either a TRUE location cache or FAKE location cache
    for key in neighbours_dictionary:

        #values are the car position IDs in that cache, ie the neighbours IDs' available 
        values = neighbours_dictionary[key]
        
        #if there are enough cars in that cache:
        if len(values) >= number_of_witnesses:

            #sample witnesses from the set of neighbours
            witnesses_ids = random.sample(values, number_of_witnesses)
            for witness_id in witnesses_ids:
                #we need to save the cache that the witness was sampled from, 
                #so we create a dictionary with the witness ID as the key, and its cache as the value
                witnesses_dictionary.update( {witness_id: key} )

        else:
            witnesses_dictionary == None
            print('None witnesses')
    #if the neighbours_dictionary has both TRUE and FAKE caches, the size of the witness_dict will be 2xnumber_of_witnesses
    #so:
    #if the witness dictionary has too many items, sample to get it to the size of number_of_witnesses
    if (len(list(witnesses_dictionary.items())) > number_of_witnesses) and (len(list(witnesses_dictionary.items())) > 0):
        witnesses_dictionary = dict(random.sample(list(witnesses_dictionary.items()), number_of_witnesses))

    return witnesses_dictionary

witness_dict = witness_dictionary_gen(neighbours_dictionary, 2)
print(witness_dict)

def get_car_from_dict(witnesses_dictionary:dict, containers:Containers):
    witnesses = []
    for id in witnesses_dictionary:
        #print(id, witnesses_dictionary[id])
        if witnesses_dictionary[id] == LocationCacheType.TRUE:
            witness = containers.get_car_from_position_index(id, containers.true_car_container_dictionary)

        else:
            witness = containers.get_car_from_position_index(id, containers.fake_car_container_dictionary)
        witnesses.append(witness)
    return witnesses

witnesses = get_car_from_dict(witness_dict, containers)
print(witnesses)

In [ ]:
#TODO: neighbours_dict has the following struct:
#TRUE CACHE: [position ids]
#FAKE CACHE: [position ids]

def is_car_a_neighbour(node, neighbours_dict):
    if node.honest is True:
        if node.true_position_index in neighbours_dict[LocationCacheType.TRUE]:
            return True
        else:
            return False

    elif node.honest is False:
        if node.fake_position_index in neighbours_dict[LocationCacheType.FAKE]:
            return True
        elif node.true_position_index in neighbours_dict[LocationCacheType.TRUE]:
            return True
        else:
            return False
        
print(neighbours_dictionary)
print(cars[2].honest)
print(cars[10].honest, cars[10].true_position_index, cars[10].fake_position_index)
boolean = is_car_a_neighbour(cars[10], neighbours_dictionary)
print(boolean)

In [ ]:
from t_pop.collections.adapters.location_cache import LocationCacheAdapter

def checks(child, named_cars:set, number_of_witnesses_needed:int, threshold:float, locations:LocationCacheAdapter) -> bool:
    """checks called from the child with respect to the parent node, to ensure that 
    all criteria for T-PoP are met."""

    parent = child.parent
    #TODO: fix get neighbour list function because i changed
    parent_neighbours = get_neighbours_dict(parent, locations)
    
    if (
    #checking the parent is a neighbour of the child
    is_car_a_neighbour(child, parent_neighbours) and
    
    #checking the child has not been named before
    child.car_id not in named_cars and
    
    #checking the parent has named enough witnesses (ie children)
    len(parent.children) >= int(number_of_witnesses_needed * threshold) and

    #checking that there is no repeats in the named witnesses (ie children) 
    len(parent.children) == len(set(parent.children))
    ):
        named_cars.add(child.car_id)
        return True
    else:
        print('checks is False')
        return False
    
tree = Tree(cars[2], depth, number_of_witnesses_per_depth, location_adapter, containers)


In [ ]:
def TPoP(tree:Tree, threshold:float, witness_number_per_depth:list, locations:LocationCacheAdapter) -> bool:
    
    named_cars = set()

    verifiedCars = [[True for car in l] for l in tree.nodes]
    
    for level in range(tree.depth - 1, -1, -1):
        number_of_witnesses_needed = witness_number_per_depth[level]
        counterDepth = 0
        indexChild = 0
        for indexParent, parent in enumerate(tree.nodes[level]):
                counterChildren = 0
            
                for child in parent.children:
                    
                    if checks(child, named_cars, number_of_witnesses_needed, threshold, locations) and verifiedCars[level + 1][indexChild]:
                        counterChildren += 1
                        counterDepth += 1
                    indexChild += 1
                    

                if counterChildren < threshold*witness_number_per_depth[level+1]:
                    verifiedCars[level][indexParent] = False
                
        if counterDepth < threshold*witness_number_per_depth[level+1]:
            tree.prover.algorithm_honesty_output = False
            
            #print('tpop output',tree.prover.algorithm_honesty_output, 'prover honesty ',tree.prover.honest)
            #return False
        else:
            tree.prover.algorithm_honesty_output = True
    #print('tpop output', tree.prover.algorithm_honesty_output, 'prover honesty ',tree.prover.honest)
    
    #return True
TPoP(tree, threshold, number_of_witnesses_per_depth, location_adapter)

In [ ]:
cars, location_adapter, containers = sim.simulator_setup(environment_size, number_of_cars, prob_coerced, prob_honest)
for car in cars:
    tree = sim.Tree(car, depth, number_of_witnesses_per_depth, location_adapter, containers)
    sim.TPoP(tree, threshold, number_of_witnesses_per_depth, location_adapter)
    if car.honest is False:
        print(car.algorithm_honesty_output)
    


In [ ]:
True_Positive = 0
True_Negative = 0
False_Positive = 0
False_Negative = 0


for car in cars:

    if car.honest is True and car.algorithm_honesty_output is True:
        True_Positive += 1
    if car.honest is True and car.algorithm_honesty_output is False:
        False_Negative += 1
    if car.honest is False and car.algorithm_honesty_output is True:
        False_Positive += 1
    if car.honest is False and car.algorithm_honesty_output is False:
        True_Negative += 1
print( 'TP', True_Positive, 'TN ',True_Negative, 'FN', False_Negative, 'FP', False_Positive)

In [ ]:
sim_df = sim.simulator_caller(number_of_simulations, number_of_cars, prob_coerced, prob_honest, depth,
                            cars, containers, location_adapter, 
                            number_of_witnesses_per_depth, density, threshold, environment_size)

In [ ]:
sim_df

In [ ]:
target_path = '/simulations/depth_' + str(depth) + '/' + 'neighbours_' + str(number_of_witnesses_per_depth[depth]) + '/'

path = sim.make_directory(target_path)


simulation_id = 0

f = open("simulation_pathfiles_depth2_neighbours_2.txt", "w+")


for probability_honest in range(0, 110, 10):
    probability_honest = probability_honest/100

    for probability_coerced in range(0, 110, 10):
        
        probability_coerced = probability_coerced/100
    
        number_of_simulations = 1

        simulation_id += 1

        sim_df = sim.simulator_caller(number_of_simulations, number_of_cars, prob_coerced, prob_honest, depth, cars, containers, location_adapter, number_of_witnesses_per_depth, density, threshold, environment_size)
        sim_pathfile = sim.save_simulation(sim_df, path, str(simulation_id) + 'p_honest' + str(probability_honest*100) + 'p_coerced' + str(probability_coerced*100)) 

        #saving the pathfiles of each simulation in the following text file.
        print('saving sim: ', str(simulation_id) + 'p_honest' + str(probability_honest*100) + 'p_coerced' + str(probability_coerced*100))
        f=open("simulation_pathfiles_depth2_neighbours_2.txt", "a+")
        f.write(sim_pathfile+'\n')
        f.close()

In [ ]:
#showcase trajectory of the platoon. 3 or 4 images. 

#In order not to disclose the sensitive information of the actual volume of traffic served by the mobile network operator, we normalize all traffic by a same random value. Therefore, traffic is not provided with a specific unit (e.g., bytes) but it is still fully comparable across space and time.
